In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('results/master_file.csv', sep=';')

In [4]:
df.shape

(176570, 12)

In [5]:
df.drop('Request ID', axis=1, inplace=True)
df.drop('Test case name', axis=1, inplace=True)
df.drop('Test suite name', axis=1, inplace=True)

In [6]:
df.columns

Index(['Endpoint', 'Method', 'Status Code', 'Start time',
       'Duration till headers received[ms]', 'Duration till body received[ms]',
       'Request Body Length', 'Response Body Length', 'Test Result'],
      dtype='object')

In [7]:
df.describe(include='all')

,Endpoint,Method,Status Code,Start time,Duration till headers received[ms],Duration till body received[ms],Request Body Length,Response Body Length,Test Result
count,176570,176570,176570.000000,176570,176570.000000,176570.000000,176570.000000,176570.000000,176570
unique,41,4,NaN,176570,NaN,NaN,NaN,NaN,2
top,/g/user,POST,NaN,2023-04-03 15:14:23.030,NaN,NaN,NaN,NaN,PASS
freq,55996,84716,NaN,1,NaN,NaN,NaN,NaN,173285
mean,NaN,NaN,229.567203,NaN,644.561149,737.753746,113.169259,615.292734,NaN
std,NaN,NaN,71.643950,NaN,2055.052325,3232.821377,216.993416,1072.367875,NaN
min,NaN,NaN,200.000000,NaN,0.000000,2.000000,0.000000,0.000000,NaN
25%,NaN,NaN,200.000000,NaN,68.000000,73.000000,0.000000,24.000000,NaN
50%,NaN,NaN,200.000000,NaN,121.000000,126.000000,36.000000,179.000000,NaN
75%,NaN,NaN,200.000000,NaN,460.000000,472.000000,83.000000,997.000000,NaN


In [8]:
# Max response time
df.iloc[df['Duration till body received[ms]'].idxmax()]

Endpoint                              /g/metric/camerabandwidth
Method                                                      GET
Status Code                                                 200
Start time                              2023-04-11 22:05:12.339
Duration till headers received[ms]                       167193
Duration till body received[ms]                          167197
Request Body Length                                           0
Response Body Length                                       8899
Test Result                                                PASS
Name: 13374, dtype: object

# Median reponse time for PASS and FAIL

In [9]:
avg_time_by_result = df.groupby('Test Result')['Duration till body received[ms]'].median()
print(avg_time_by_result)

Test Result
FAIL    162.0
PASS    126.0
Name: Duration till body received[ms], dtype: float64


In [10]:
# Group the DataFrame by endpoint and Method, and calculate the median response time for each group
grouped = df.groupby(['Endpoint', 'Method', 'Test Result']).agg({'Duration till body received[ms]': 'median'}).reset_index()

# Filter the grouped DataFrame to include only PASS and FAIL results
grouped_pass = grouped[grouped['Test Result'] == 'PASS']
grouped_fail = grouped[grouped['Test Result'] == 'FAIL']

# Merge the two DataFrames
result = pd.merge(grouped_pass, grouped_fail, on=['Endpoint', 'Method'], suffixes=('_pass', '_fail'))

# Rename the column headers to be more concise
result = result.rename(columns={'Test Result_pass': 'Result', 'Test Result_fail': 'Result','Duration till body received[ms]_pass': 'Pass_median', 'Duration till body received[ms]_fail': 'Fail_median'})
print(result)

                     Endpoint  Method Result  Pass_median Result  Fail_median
0           /asset/list/image     GET   PASS         55.0   FAIL      10026.0
1      /asset/prev/image.jpeg     GET   PASS        195.5   FAIL        124.0
2         /g/aaa/authenticate    POST   PASS         65.0   FAIL         63.0
3            /g/aaa/authorize    POST   PASS        681.0   FAIL        146.0
4      /g/aaa/change_password    POST   PASS         98.0   FAIL         93.0
5      /g/aaa/forgot_password    POST   PASS        375.0   FAIL        763.5
6               /g/aaa/logout    POST   PASS         82.0   FAIL         75.0
7             /g/cameratunnel     PUT   PASS       4030.0   FAIL      63366.0
8                   /g/device  DELETE   PASS        673.0   FAIL        796.0
9                   /g/device     GET   PASS        185.0   FAIL        245.0
10                  /g/device     PUT   PASS       1170.0   FAIL      33245.0
11                  /g/layout  DELETE   PASS        120.0   FAIL

# Errors of each end point

#### /g/aaa/authenticate

In [32]:
# Filter the data to include only rows where the result column is 'failure'
# and the endpoint name column is '/aaa/name'
filtered_df = df[(df['Test Result'] == 'FAIL') & (df['Status Code'] != 200)]
                # & (df['Method'] == 'PUT')]
                # & (df['Endpoint'] == '/g/device')]

# Group the filtered data by the status code column and count the number of occurrences of each status code
#unique_status_codes = filtered_df.groupby('Status Code').size()

# Group the filtered data by the endpoint name and status code columns, and count the number of occurrences of each status code
grouped_df = filtered_df.groupby(['Endpoint', 'Status Code']).size().reset_index(name='count')

# Print the unique status codes and their counts for the specified endpoint name
print(f"Status codes for endpoint '/g/aaa/authenticate':")
for index, row in grouped_df.iterrows():
    print(f"Status code {row['Status Code']}: {row['count']} occurrences")

example_row = filtered_df.iloc[0]
print (example_row)

Status codes for endpoint '/g/aaa/authenticate':
Status code 202: 13 occurrences
Status code 412: 1 occurrences
Status code 204: 152 occurrences
Status code 403: 1 occurrences
Status code 409: 5 occurrences
Status code 400: 2 occurrences
Endpoint                                        /g/aaa/logout
Method                                                   POST
Status Code                                               204
Start time                            2023-03-28 21:06:33.704
Duration till headers received[ms]                         54
Duration till body received[ms]                            58
Request Body Length                                         2
Response Body Length                                        0
Test Result                                              FAIL
Name: 2587, dtype: object
